In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from pyearth import Earth
from sklearn.impute import KNNImputer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from statsmodels.tools.tools import add_constant
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import time

import warnings
warnings.filterwarnings("ignore")

## Prophet Feature Selection, so Dropping All Lag and Rolling

In [2]:
covid_df = pd.read_csv('ohio_multi_df.csv', parse_dates=['Date'])
covid_df.drop(columns = ['PROVINCE_STATE_NAME'], inplace = True)
covid_df = covid_df.rename(columns={'Date': 'ds', 'PEOPLE_POSITIVE_NEW_CASES_COUNT': 'y'})
covid_df = covid_df.drop(['new_cases_1 lag', 'new_cases_2_lag', 'new_cases_3_lag', 'new_cases_4_lag', 'new_cases_7_lag', 'new_cases_30_lag', 'new_cases_50_lag', 'new_cases_150_lag', 'new_cases_350_lag', 'new_deaths_1_lag', 'new_deaths_2_lag', 'new_deaths_3_lag', 'new_deaths_4_lag', 'new_deaths_7_lag', 'new_deaths_30_lag', 'new_deaths_50_lag'], axis=1)

train_size = int(len(covid_df)* 0.85)
covid_train = covid_df[0:train_size] # notice here the original test test is never used

In [3]:
train_size

671

In [4]:
X_train = covid_train.drop(columns=['ds', 'y'], axis=1)
y_train = covid_train['y']

In [5]:
model_a = CatBoostRegressor().fit(X_train, y_train, verbose=False)
predictors_1 = pd.concat([pd.Series(model_a.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_1[1].to_list())

['new_cases_10_max', 'new_cases_10_mean', 'new_hospitalized_patients', 'new_cases_10_std', 'current_intensive_care_patients', 'new_cases_50_std', 'PEOPLE_DEATH_NEW_COUNT', 'average_temperature_celsius', 'new_cases_10_min', 'Series_Complete_5Plus', 'Series_Complete_18Plus', 'dew_point', 'vaccination_rate_complete', 'relative_humidity', 'new_deaths_10_mean', 'rainfall_mm', 'Administered_Dose1_Recip_18Plus', 'new_cases_50_mean', 'minimum_temperature_celsius', 'maximum_temperature_celsius', 'new_cases_100_std', 'new_cases_100_max', 'new_deaths_100_std', 'new_deaths_100_mean', 'new_cases_100_min', 'vaccination_rate_1dose', 'Administered_Dose1_Recip_12Plus', 'Administered_Dose1_Recip_5Plus', 'Booster_Doses', 'new_cases_50_max']


In [6]:
model_b = LGBMRegressor().fit(X_train, y_train, verbose=False)
predictors_2 = pd.concat([pd.Series(model_b.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_2[1].to_list())

['new_cases_10_std', 'new_hospitalized_patients', 'relative_humidity', 'new_cases_10_mean', 'new_cases_10_max', 'new_cases_50_std', 'PEOPLE_DEATH_NEW_COUNT', 'rainfall_mm', 'current_intensive_care_patients', 'maximum_temperature_celsius', 'new_cases_10_min', 'new_deaths_10_mean', 'new_cases_50_mean', 'dew_point', 'average_temperature_celsius', 'new_deaths_10_std', 'minimum_temperature_celsius', 'new_cases_100_std', 'new_deaths_50_std', 'new_deaths_50_mean', 'new_deaths_100_mean', 'Administered_Dose1_Recip', 'Booster_Doses_50Plus', 'new_deaths_10_max', 'Booster_Doses', 'Administered_Dose1_Recip_65Plus', 'new_deaths_100_std', 'new_cases_100_mean', 'Administered_Dose1_Recip_5Plus', 'Booster_Doses_65Plus']


In [7]:
from sklearn.ensemble import GradientBoostingRegressor,GradientBoostingClassifier, BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,AdaBoostClassifier

model_c = GradientBoostingRegressor().fit(X_train, y_train)
predictors_3 = pd.concat([pd.Series(model_c.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_3[1].to_list())

['new_hospitalized_patients', 'new_cases_10_mean', 'new_cases_10_std', 'new_cases_10_max', 'maximum_temperature_celsius', 'new_cases_10_min', 'new_cases_50_std', 'dew_point', 'new_cases_100_std', 'PEOPLE_DEATH_NEW_COUNT', 'average_temperature_celsius', 'new_cases_50_mean', 'Booster_Doses_65Plus', 'new_deaths_10_mean', 'new_deaths_50_mean', 'relative_humidity', 'current_intensive_care_patients', 'new_deaths_100_std', 'minimum_temperature_celsius', 'Administered_Dose1_Recip', 'Series_Complete_65Plus', 'Series_Complete_Yes', 'new_deaths_100_mean', 'rainfall_mm', 'Completeness_pct', 'new_deaths_50_std', 'new_cases_100_min', 'vaccination_rate_1dose', 'Booster_Doses_50Plus', 'new_cases_100_mean']


In [8]:
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier

model_d = AdaBoostRegressor(random_state=1, base_estimator = DecisionTreeRegressor(random_state=1)).fit(X_train, y_train)
predictors_4 = pd.concat([pd.Series(model_d.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_4[1].to_list())

['new_cases_10_mean', 'new_cases_10_std', 'new_hospitalized_patients', 'new_cases_50_std', 'new_cases_10_max', 'new_cases_10_min', 'new_cases_50_max', 'PEOPLE_DEATH_NEW_COUNT', 'minimum_temperature_celsius', 'rainfall_mm', 'dew_point', 'average_temperature_celsius', 'current_intensive_care_patients', 'maximum_temperature_celsius', 'new_deaths_10_mean', 'new_deaths_50_std', 'new_deaths_100_std', 'new_deaths_100_mean', 'new_cases_100_std', 'new_cases_50_mean', 'relative_humidity', 'new_cases_100_mean', 'new_deaths_10_std', 'new_deaths_100_max', 'new_cases_100_max', 'Administered_Dose1_Recip_18Plus', 'Administered_Dose1_Recip_65Plus', 'Series_Complete_12Plus', 'Booster_Doses_18Plus', 'Administered_Dose1_Recip']


In [9]:
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier

model_e = RandomForestRegressor().fit(X_train, y_train)
predictors_5 = pd.concat([pd.Series(model_e.feature_importances_), pd.Series(X_train.columns)], axis=1).sort_values(by=0, ascending = False).head(30)
print(predictors_5[1].to_list())

['new_hospitalized_patients', 'new_cases_10_mean', 'new_cases_10_max', 'new_cases_10_std', 'new_cases_50_std', 'new_cases_10_min', 'maximum_temperature_celsius', 'minimum_temperature_celsius', 'dew_point', 'rainfall_mm', 'new_cases_100_std', 'Series_Complete_12Plus', 'PEOPLE_DEATH_NEW_COUNT', 'average_temperature_celsius', 'relative_humidity', 'Booster_Doses_18Plus', 'new_deaths_10_mean', 'new_deaths_100_std', 'Booster_Doses_50Plus', 'new_cases_50_mean', 'Series_Complete_65Plus', 'current_intensive_care_patients', 'Administered_Dose1_Recip_65Plus', 'new_deaths_100_mean', 'new_deaths_10_std', 'Administered_Dose1_Recip_18Plus', 'Series_Complete_Yes', 'new_cases_100_mean', 'Administered_Dose1_Recip_5Plus', 'Administered_Dose1_Recip']


In [10]:
# using MARS summary for feature selection
model = Earth(max_terms=1000, max_degree=1, feature_importance_type = 'rss') # note, terms in brackets are the hyperparameters 
model.fit(X_train,y_train)
print(model.summary())

Earth Model
--------------------------------------------------------------
Basis Function                           Pruned  Coefficient  
--------------------------------------------------------------
(Intercept)                              No      -71582.2     
h(new_cases_10_mean-10954.8)             Yes     None         
h(10954.8-new_cases_10_mean)             No      3.68858      
h(new_cases_50_mean-8551.92)             No      20.2323      
h(8551.92-new_cases_50_mean)             No      -27.2946     
h(new_cases_10_std-5096.76)              No      14.502       
h(5096.76-new_cases_10_std)              No      3.89783      
h(current_intensive_care_patients-1216)  Yes     None         
h(1216-current_intensive_care_patients)  Yes     None         
new_hospitalized_patients                No      4.68497      
h(new_cases_10_max-20917)                No      2.43951      
h(20917-new_cases_10_max)                No      -1.62331     
h(Booster_Doses_65Plus-885508)           No

In [11]:
def common_member(a, b, c, d, e):
    set_1 = set(a)
    set_2 = set(b)
    set_3 = set(c)
    set_4 = set(d)
    set_5 = set(e)
 
    if (set_1 & set_2 & set_3 & set_4 & set_5):
        print(set_1 & set_2 & set_3 & set_4 & set_5)


In [12]:
a = predictors_1[1].to_list()
b = predictors_2[1].to_list()
c = predictors_3[1].to_list()
d = predictors_4[1].to_list()
e = predictors_5[1].to_list()

In [13]:
full = common_member(a, b, c, d, e)
full

{'current_intensive_care_patients', 'new_cases_10_min', 'relative_humidity', 'dew_point', 'rainfall_mm', 'new_deaths_100_mean', 'new_cases_50_std', 'maximum_temperature_celsius', 'new_cases_10_max', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_100_std', 'new_cases_50_mean', 'new_deaths_10_mean', 'minimum_temperature_celsius', 'new_cases_10_mean', 'new_cases_10_std', 'average_temperature_celsius', 'new_hospitalized_patients', 'new_deaths_100_std'}


## XGBoost Feature Selection, includes lag and rolling

In [14]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from pyearth import Earth
from sklearn.impute import KNNImputer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from statsmodels.tools.tools import add_constant
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import time

import warnings
warnings.filterwarnings("ignore")

In [15]:
covid_df = pd.read_csv('ohio_multi_df.csv', parse_dates=['Date'])
covid_df.drop(columns = ['PROVINCE_STATE_NAME'], inplace = True)
covid_df = covid_df.rename(columns={'Date': 'ds', 'PEOPLE_POSITIVE_NEW_CASES_COUNT': 'y'})

train_size = int(len(covid_df)* 0.85)
covid_train = covid_df[0:train_size]

X_train = covid_train.drop(columns=['ds', 'y'], axis=1)
X_train = X_train.astype('float32')
y_train = covid_train['y']

In [16]:
model_a = CatBoostRegressor().fit(X_train, y_train, verbose=False)
predictors_7 = pd.concat([pd.Series(model_a.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(40)
print(predictors_7[1].to_list())

['new_cases_10_mean', 'current_intensive_care_patients', 'new_hospitalized_patients', 'new_cases_10_max', 'new_cases_7_lag', 'new_cases_50_std', 'new_cases_2_lag', 'new_cases_1 lag', 'new_cases_3_lag', 'new_cases_10_std', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_30_lag', 'new_cases_10_min', 'new_cases_4_lag', 'new_deaths_3_lag', 'rainfall_mm', 'Series_Complete_Yes', 'maximum_temperature_celsius', 'Series_Complete_18Plus', 'new_deaths_100_std', 'Booster_Doses_50Plus', 'new_cases_350_lag', 'new_cases_50_mean', 'new_cases_100_min', 'new_deaths_1_lag', 'new_deaths_10_std', 'Booster_Doses', 'Booster_Doses_65Plus', 'new_deaths_2_lag', 'new_deaths_4_lag', 'Administered_Dose1_Recip_18Plus', 'Booster_Doses_18Plus', 'new_deaths_100_mean', 'relative_humidity', 'minimum_temperature_celsius', 'dew_point', 'new_deaths_50_std', 'new_cases_150_lag', 'new_deaths_7_lag', 'Series_Complete_65Plus']


In [17]:
model_b = LGBMRegressor().fit(X_train, y_train, verbose=False)
predictors_8 = pd.concat([pd.Series(model_b.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_8[1].to_list())

['new_cases_7_lag', 'new_cases_10_std', 'new_cases_1 lag', 'new_cases_10_max', 'new_hospitalized_patients', 'new_cases_4_lag', 'new_cases_10_mean', 'new_cases_3_lag', 'new_cases_2_lag', 'relative_humidity', 'new_cases_10_min', 'new_cases_30_lag', 'PEOPLE_DEATH_NEW_COUNT', 'new_deaths_30_lag', 'new_cases_50_std', 'current_intensive_care_patients', 'new_deaths_7_lag', 'new_deaths_2_lag', 'new_deaths_50_lag', 'new_deaths_4_lag', 'rainfall_mm', 'new_cases_100_std', 'new_cases_350_lag', 'new_deaths_1_lag', 'new_deaths_3_lag', 'new_deaths_10_std', 'new_deaths_10_mean', 'new_cases_50_lag', 'new_cases_150_lag', 'maximum_temperature_celsius']


In [18]:
model_g = xgb.XGBRegressor().fit(X_train, y_train, verbose=False)
predictors_6 = pd.concat([pd.Series(model_g.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_6[1].to_list())

['new_hospitalized_patients', 'new_cases_10_max', 'new_cases_10_mean', 'new_cases_10_std', 'new_cases_7_lag', 'new_cases_10_min', 'new_cases_1 lag', 'maximum_temperature_celsius', 'new_deaths_1_lag', 'Administered_Dose1_Recip', 'new_deaths_3_lag', 'new_cases_150_lag', 'new_cases_100_std', 'average_temperature_celsius', 'new_deaths_100_mean', 'new_deaths_2_lag', 'new_cases_2_lag', 'Booster_Doses_50Plus', 'new_cases_100_mean', 'new_deaths_100_std', 'new_cases_3_lag', 'rainfall_mm', 'new_cases_50_std', 'Completeness_pct', 'new_deaths_10_std', 'dew_point', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_50_lag', 'new_cases_350_lag', 'new_deaths_10_mean']


In [19]:
model = Earth(max_terms=1000, max_degree=1, feature_importance_type = 'rss', allow_missing=True) 
model.fit(X_train,y_train)
print(model.summary())

Earth Model
------------------------------------------------------------------------
Basis Function                                     Pruned  Coefficient  
------------------------------------------------------------------------
(Intercept)                                        No      -118393      
h(new_cases_10_mean-10954.8)                       No      -4.53625     
h(10954.8-new_cases_10_mean)                       Yes     None         
present(new_cases_1 lag)                           Yes     None         
missing(new_cases_1 lag)                           Yes     None         
h(new_cases_1 lag-49)*present(new_cases_1 lag)     No      -0.59809     
h(49-new_cases_1 lag)*present(new_cases_1 lag)     No      24.0697      
h(new_cases_50_mean-8551.92)                       No      31.9768      
h(8551.92-new_cases_50_mean)                       No      0.567102     
present(new_cases_3_lag)                           Yes     None         
missing(new_cases_3_lag)               

In [20]:
def common_member(a, b, c):
    set_1 = set(a)
    set_2 = set(b)
    set_3 = set(c)
 
    if (set_1 & set_2 & set_3):
        print(set_1 & set_2 & set_3)


In [21]:
a = predictors_7[1].to_list()
b = predictors_8[1].to_list()
c = predictors_6[1].to_list()

In [22]:
full = common_member(a, b, c)
full

{'new_cases_10_min', 'new_cases_1 lag', 'new_cases_2_lag', 'rainfall_mm', 'new_cases_350_lag', 'new_cases_50_std', 'maximum_temperature_celsius', 'new_deaths_10_std', 'new_cases_3_lag', 'new_deaths_3_lag', 'new_cases_10_std', 'new_cases_7_lag', 'PEOPLE_DEATH_NEW_COUNT', 'new_deaths_2_lag', 'new_cases_10_mean', 'new_deaths_1_lag', 'new_cases_150_lag', 'new_hospitalized_patients', 'new_cases_10_max'}
